https://github.com/zhinst/blogs/blob/master/Hands-on%20Superconducting%20Qubit%20Characterization/Single-Qubit-Tuneup.md
but using SHFQA and SHFSG
https://github.com/zhinst/laboneq/blob/main/examples/03_superconducting_qubits/00_qubit_tuneup_shfsg_shfqa_shfqc.ipynb

This Script is for: 1. General Explanation, 2. Qubit and Readout Resonator Spectroscopy and 3. Pulsed Qubit Control and Rabi Oscillations

1. General Explanation
What do I need for a SINGLE Qubit Measurement. Hardware: SHFQC, Software: LabOne Q


In [ ]:
#Hardware Setup
from IPython.display import Image
Image(filename='ZI_Image1.png')

In [ ]:
#Further Explanation Setup
from IPython.display import Image
Image(filename='ZI_Image2.png')

Explanation Spectroscopy
1. Resonator Spectroscopy
2. Qubit Spectroscopy

Resonator Spectroscopy:

In [ ]:
from IPython.display import Image
Image(filename='ZI_Image3.png')

2. Qubit Spectroscopy Setup

In [ ]:
from IPython.display import Image
Image(filename='ZI_Image5.png')

In [ ]:
from IPython.display import Image
Image(filename='ZI_Image4.png')

Roles of Readout Power

In [ ]:
from IPython.display import Image
Image(filename='ZI_Image6.png')

In the following image you can see the Software Architecture for a bigger setup

In [ ]:
from IPython.display import Image
Image(filename='flowchart_QCCS.png')

The Code for reading out a single Qubit:

1. Imports

In [ ]:
#Initial Imports
%config IPCompleter.greedy=True

# convenience import for all QCCS software functionality
from laboneq.simple import *

# helper import - needed to extract qubit and readout parameters from measurement data
from tuneup_helper import *

# plotting and fitting functionality
from laboneq.analysis.fitting import (
    lorentzian,
    oscillatory,
    oscillatory_decay,
)

# for saving results and pulse sheets
from pathlib import Path
import time

import matplotlib.pyplot as plt
import numpy as np

Create Device Setup

In [ ]:
#DESCRITPTOR 
#Define initial PHYSICAL device setup, Logical Signal Lines of the Instrument are defined here and later in a Signal Map mapped to the Experimental Signal lines 

#Fill in missing address

descriptor_setup_small= """\
instruments:
  SHFSG:
  - address: DEV12265
    uid: device_shfsg_1
  SHFQA:
  - address: DEV12249
    uid: device_shfqa_1
  PQSC:
  - address: DEV10----
    uid: device_pqsc
connections:
  device_shfsg:
    - iq_signal: q0/drive_line
      ports: SGCHANNELS/0/OUTPUT
  device_shfqa:
    - iq_signal: q0/measure_line
      ports: QACHANNELS/0/OUTPUT
    - acquire_signal: q0/acquire_line
      ports: QACHANNELS/0/INPUT
  device_pqsc:
    - to: device_shfsg_1
      port: ZSYNCS/0
    - to: device_shfqa_1
      port: ZSYNCS/1
"""
#Example to understand better:
"""  device_shfqa:
    - iq_signal: q0/measure_line
      ports: QACHANNELS/0/OUTPUT
    - acquire_signal: q0/acquire_line
      ports: QACHANNELS/0/INPUT
    - iq_signal: q1/measure_line
      ports: QACHANNELS/0/OUTPUT
    - acquire_signal: q1/acquire_line
      ports: QACHANNELS/0/INPUT
q0 and q1 are signal groups, but use the same physical 
Here, we have four logical signal lines, two named measure_line, which are iq_signal types 
for sending qubit readout pulses, and two named acquire_line, for data acquisition on the input of SHFQA. 
Two of the measure and acquire lines are in the logical group q0 and the others are in q1. As both line pairs use 
the same physical channel on the instrument, the output signals are added, resulting in a frequency multiplexing.
 On the input side, each logical signal line has its own integration weights."""

In [ ]:
# define the DeviceSetup from descriptor - additionally include information on the dataserver used to connect to the instruments 
#with the device set
device_setup = DeviceSetup.from_descriptor(
    #or with example import and yaml also possible: yaml_text=descriptor_shfsg_shfqa_pqsc
    descriptor_setup_small,
    server_host="192.168.1.10", #IP-Address of the LabOne (NOT Q) dataserver used to communicate with the instrument
    server_port="8004",
    setup_name="psi",
) 

# define shortcut to logical signals for convenience
lsg = {
    qubit_name: device_setup.logical_signal_groups[qubit_name].logical_signals
    for qubit_name in device_setup.logical_signal_groups.keys()
}

Qubit Parameters

A python dictionary containing all parameters needed to control and readout the qubits - frequencies, pulse lengths, timings

May initially contain only the design parameters and will be updated with measurement results during the tuneup procedure

In [ ]:
"""
# a function to define a collection of single qubit control and readout parameters as a python dictionary
def single_qubit_parameters():
    return {
        "freq": 100e6,  # qubit 0 drive frequency in [Hz] - relative to local oscillator for qubit drive upconversion
        "ro_freq": 5e6,  # 50e6,
        "ro_delay": 0,  # 15e-9,#100e-9,
        "ro_int_delay": 0,  # 40-9,
        "qb_len_spec": 1e-6,
        "qb_len": 700e-9,
        "qb_amp_spec": 1.0,
        "pi_amp": 1,
        "freq_ef": -500e6,
    }


# for sake of simplicity, give all qubits the same initial parameters
qubit_parameters = {
    k: single_qubit_parameters() for k in device_setup.logical_signal_groups.keys()
}


# up / downconversion settings - to convert between IF and RF frequencies
def single_lo_settings():
    return {
        # SHFQA LO Frequency
        "shfqa_lo": 8.0e9,
        # SHFSG LO Frequencies, one center frequency per two channels on SHFQC
        "shfsg_lo": 5.0e9,
    }


lo_settings = {
    k: single_lo_settings() for k in device_setup.logical_signal_groups.keys()
}
"""
# a collection of qubit control and readout parameters as a python dictionary
qubit_parameters = {
    'ro_freq' :  10e6,           # readout frequency of qubit 0 in [Hz] - relative to local oscillator for readout drive upconversion
    'ro_amp' : 0.5,              # readout amplitude
    'ro_amp_spec': 0.05,         # readout amplitude for spectroscopy
    'ro_len' : 1.0e-6,           # readout pulse length in [s]
    'ro_len_spec' : 1.0e-6,      # readout pulse length for resonator spectroscopy in [s]
    'ro_delay': 100e-9,          # readout delay after last drive signal in [s]
    'ro_int_delay' : 180e-9,     # readout line offset calibration - delay between readout pulse and start of signal acquisition in [s]
    
    'qb_freq': 20e6,             # qubit 0 drive frequency in [Hz] - relative to local oscillator for qubit drive upconversion
    'qb_amp_spec': 0.01,         # drive amplitude of qubit spectroscopy
    'qb_len_spec': 15e-6,        # drive pulse length for qubit spectroscopy in [s]
    'qb_len' : 4e-7,             # qubit drive pulse length in [s]
    'pi_amp' : 0.5,              # qubit drive amplitude for pi pulse
    'pi_half_amp' : 0.25,        # qubit drive amplitude for pi/2 pulse
    'qb_t1' : 100e-6,            # qubit T1 time
    'qb_t2' : 100e-6,            # qubit T2 time
    'relax' : 200e-6             # delay time after each measurement for qubit reset in [s]
}

# up / downconversion settings - to convert between IF and RF frequencies
lo_settings = {
    'qb_lo': 4.0e9,              # qubit LO frequency in [Hz]
    'ro_lo': 7.0e9               # readout LO frequency in [Hz]
}


In [ ]:
qubit_parameters['ro_int_delay']= 350e-9

Calibration

In [ ]:
"""
# function that defines the device settings for qubit and readout parameters
def define_calibration(device_setup, parameters, lo_settings):
    # Define LOs
    def single_oscillator(id, qubit, lo_type):
        oscillator = Oscillator()
        oscillator.uid = f"{id}" + f"{qubit}" + "_osc"
        oscillator.frequency = lo_settings[qubit][lo_type]
        return oscillator

    readout_lo_dict = {
        k: single_oscillator("readout_lo_", k, "shfqa_lo")
        for k in device_setup.logical_signal_groups.keys()
    }

    drive_lo_dict = {
        k: single_oscillator("drive_lo_", k, "shfsg_lo")
        for k in device_setup.logical_signal_groups.keys()
    }

    # the calibration object will later be applied to the device setup
    calibration = Calibration()

    # qubits q0-q3 are multiplexed on one acquisition line
    calibration[
        device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"]
    ] = SignalCalibration(
        oscillator=Oscillator(
            frequency=parameters["q0"]["ro_freq"],
            modulation_type=ModulationType.SOFTWARE,
        ),
        # add an offset between the readout pulse and the start of the data acquisition - to compensate for round-trip time of readout pulse
        port_delay=parameters["q0"]["ro_delay"] + parameters["q0"]["ro_int_delay"],
        local_oscillator=readout_lo_dict["q0"],
        range=5,
    )

    for logical_signal_group in device_setup.logical_signal_groups.keys():
        # measure line
        calibration[
            device_setup.logical_signal_groups[logical_signal_group].logical_signals[
                "measure_line"
            ]
        ] = SignalCalibration(
            oscillator=Oscillator(
                frequency=parameters[logical_signal_group]["ro_freq"],
                modulation_type=ModulationType.SOFTWARE,
            ),
            port_delay=parameters["q0"]["ro_delay"],
            local_oscillator=readout_lo_dict["q0"],
            range=5,
        )

        calibration[
            device_setup.logical_signal_groups[logical_signal_group].logical_signals[
                "drive_line"
            ]
        ] = SignalCalibration(
            # each logical signal can have an oscillator associated with it
            oscillator=Oscillator(
                frequency=parameters[logical_signal_group]["freq"],
                modulation_type=ModulationType.HARDWARE,
            ),
            # DANGER! Verify which qubits share the same LOs!
            local_oscillator=drive_lo_dict[logical_signal_group],
            range=5,
        )
    return calibration

"""
# function that defines a setup calibration containing the qubit / readout parameters 
def define_calibration(parameters):

     # the calibration object will later be applied to the device setup
    my_calibration = Calibration()

    my_calibration["/logical_signal_groups/q0/drive_line"] = \
        SignalCalibration(
           # each logical signal can have an oscillator associated with it
            oscillator=Oscillator(
                frequency=parameters['qb_freq'],
                modulation_type=ModulationType.HARDWARE
            ),
            local_oscillator=Oscillator(
                frequency=lo_settings['qb_lo'],
            ),
            range=-30
        )
    
    # readout drive line
    my_calibration["/logical_signal_groups/q0/measure_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=parameters['ro_freq'],
                modulation_type=ModulationType.SOFTWARE
            ),
            port_delay=parameters['ro_delay'],
            local_oscillator=Oscillator(
                frequency=lo_settings['ro_lo'],
            ),
           range=-30
       )
    # acquisition line
    my_calibration["/logical_signal_groups/q0/acquire_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=parameters['ro_freq'],
                modulation_type=ModulationType.SOFTWARE
            ),
            # add an offset between the readout pulse and the start of the data acquisition - to compensate for round-trip time of readout pulse 
            port_delay=parameters['ro_delay'] + parameters['ro_int_delay'],
            local_oscillator=Oscillator(
                frequency=lo_settings['ro_lo'],
            ),
            range=-30
        )
  
    return my_calibration


# define Calibration object based on qubit control and readout parameters
my_calibration = define_calibration(parameters=qubit_parameters)
# apply calibration to device setup
device_setup.set_calibration(my_calibration)


Set qubit control and readout calibration, and apply it to the device setup

In [ ]:
# define Calibration object based on qubit control and readout parameters
calibration = define_calibration(device_setup, qubit_parameters, lo_settings)
# apply calibration to device setup
device_setup.set_calibration(calibration)

Create and Connect to a QCCS Session

Establishes the connection to the instruments and readies them for experiments
QCCS monitor is a software to monitor all the devices in one. It can be started as follows:
LabOne needs to run, be connected to the server and then 

In [ ]:
# perform experiments in emulation mode only? - if True, also generate dummy data for fitting
emulate = False

# create and connect to a session
session = Session(device_setup=device_setup)
session.connect(do_emulation=emulate)

Signal Lines: LabOne Q abstracts the real, physical channels in your system into logical signal lines, which are grouped in logical signal groups. Logical signal lines abstract instruments, physical channels, and hardware calibration settings into a single Python object. This separates the hardware from the quantum experiments — which you can define freely, and whose experimental signal lines are mapped only later to the logical signal lines. This allows for instrumentation-agnostic experiment definitions, which can be re-used later.